<a href="https://colab.research.google.com/github/k-washi/3d-Bonet-Boost/blob/master/bonet_s3dis_retraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D - BoNetのスタート

データセットは、[Stanford 2D-3D-Semantics Dataset](http://buildingparser.stanford.edu/dataset.html)

h5データへの変更は、[pqhieu/jsis3d](https://github.com/pqhieu/jsis3d/blob/master/scripts/prepare_h5.py)にて提供されている。
また、データセットのh5 versionは、以下の3D-BoNetのリポジトリからダウンロードできる。


# 3D-BoNetのインストール
[Learning Object Bounding Boxes for 3D Instance Segmentation on Point Clouds](https://arxiv.org/abs/1906.01140)

https://github.com/Yang7879/3D-BoNet
---



In [0]:
!rm -rf 3D-BoNet

In [2]:
!git clone https://github.com/k-washi/3D-BoNet.git

Cloning into '3D-BoNet'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 295 (delta 87), reused 66 (delta 32), pack-reused 151
Receiving objects: 100% (295/295), 72.02 MiB | 33.23 MiB/s, done.
Resolving deltas: 100% (149/149), done.


# 補助パッケージの取得

https://github.com/k-washi/3d-Bonet-Boost

In [0]:
!rm -rf 3d-Bonet-Boost

In [0]:
!git clone https://github.com/k-washi/3d-Bonet-Boost.git

Cloning into '3d-Bonet-Boost'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 66 (delta 39), reused 35 (delta 16), pack-reused 0
Unpacking objects: 100% (66/66), done.


#python ライブラリの確認

tensorflow 1.15

In [0]:
%tensorflow_version 1.x
!pip show tensorflow
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
device_name

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: google-pasta, tensorflow-estimator, numpy, six, wrapt, opt-einsum, termcolor, wheel, keras-applications, tensorboard, astor, grpcio, gast, keras-preprocessing, protobuf, absl-py
Required-by: stable-baselines, magenta, fancyimpute


'/device:GPU:0'

# データセットのダウンロード

Google Driveにデータセットを置くと仮定

/content/drive/My Drive/data/Data_S3DIS.zip

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


データを解凍

In [0]:
!apt-get -y install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
!mkdir s3dis
!unzip /content/drive/My\ Drive/data/Data_S3DIS.zip -d s3dis > NILL

# 3D-BoNet: Compile tensorflow operation by cuda 

In [0]:
!cp 3d-Bonet-Boost/cu_compile_v1.15.py 3D-BoNet/tf_ops/cu_compile.py

コンパイルに必要なコマンドを出力

In [0]:
!nvcc --version
!which nvcc

!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so*
!ls /usr/lib64-nvidia/libcuda*

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
/usr/local/cuda/bin/nvcc
lrwxrwxrwx 1 root root        29 Dec  5 22:54 /usr/lib/x86_64-linux-gnu/libcudnn.so -> /etc/alternatives/libcudnn_so
lrwxrwxrwx 1 root root        17 Oct 27 21:49 /usr/lib/x86_64-linux-gnu/libcudnn.so.7 -> libcudnn.so.7.6.5
-rw-r--r-- 1 root root 428711256 Oct 27 21:49 /usr/lib/x86_64-linux-gnu/libcudnn.so.7.6.5
/usr/lib64-nvidia/libcuda.so	/usr/lib64-nvidia/libcuda.so.418.67
/usr/lib64-nvidia/libcuda.so.1


In [0]:
!ls /usr/local/cuda-10.1/

bin	extras	 libnsight  nsightee_plugins  README   src	version.txt
compat	include  libnvvp    nvml	      samples  targets
doc	lib64	 LICENSE    nvvm	      share    tools


compileに必要なcudaディレクトリとGC上で実行することを引数で与える

In [0]:
!python 3D-BoNet/tf_ops/cu_compile.py --cuda /usr/local/cuda-10.1 --googleColab

python3: can't open file '3D-BoNet/tf_ops/cu_compile.py': [Errno 2] No such file or directory


上のpythonの出力コマンドをコピーして実行 (2020/3/9時点では、以下のコマンド)

In [3]:
!/usr/local/cuda-10.1/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!/usr/local/cuda-10.1/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_so.so -shared -fPIC -I/usr/local/lib/python3.6/dist-packages/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0 -I /usr/local/cuda-10.1/include -I/usr/local/lib/python3.6/dist-packages/tensorflow_core/include/external/nsync/public -L/usr/local/lib/python3.6/dist-packages/tensorflow_core -l:libtensorflow_framework.so.1 -lcudart -lcuda -L /usr/local/cuda-10.1/lib64 -O2
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling.cpp 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_so.so -shared -fPIC -I/usr/local/lib/python3.6/dist-packages/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0 -I /usr/local/cuda-10.1/include -I/usr/local/lib/python3.6/dist-packages/tensorflow_core/include/external/nsync/public -L/usr/local/lib/python3.6/dist-packages/tensorflow_core -l:libtensorflow_framework.so.1 -lcudart -lcuda -L /usr/local/cuda-10.1/lib64 -O2
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate.cpp -o 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate_so.so -shared -fPIC -I/usr/local/lib/python3.6/dist-packages/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0 -I /usr/local/cuda-10.1/include -I/usr/local/lib/python3.6/dist-packages/tensorflow_core/include/external/nsync/public -L/usr/local/lib/python3.6/dist-packages/tensorflow_core -l:libtensorflow_framework.so.1 -lcudart -lcuda -L /usr/local/cuda-10.1/lib64 -O2

3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp: In lambda function:
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:22:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
         c->WithRank(c->input(1), 3, &dims2);
                                            ^
In file included from 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:8:0:
/usr/local/lib/python3.6/dist-packages/tensorflow_core/include/tensorflow/core/framework/shape_inference.h:394:10: note: declared here
   Status WithRank(ShapeHandle shape, int64 rank,
          ^~~~~~~~
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp: In lambda function:
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:47:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::Infere

In [0]:
!ls 3D-BoNet/tf_ops/sampling/

__init__.py		tf_sampling.cpp   tf_sampling_g.cu.o  tf_sampling_so.so
tf_sampling_compile.sh	tf_sampling_g.cu  tf_sampling.py


# 3D-BoNet の学習を実行する

In [0]:
import sys
sys.path.append('/content/3D-BoNet/')

In [0]:
import os
%tensorflow_version 1.x 
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  ## specify the GPU to use

from main_3D_BoNet import BoNet
from helper_data_s3dis import Data_Configs as Data_Configs

In [0]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [7]:
configs = Data_Configs()
net = BoNet(configs = configs)
net.creat_folders(name='log', re_train=False)

re_train: False
./log/test_sum/ : deleted and then created!
./log/train_sum/ : deleted and then created!
./log/train_mod/ : deleted and then created!


In [0]:
from main_eval import Evaluation

In [0]:
train_areas =['Area_1', 'Area_2'  ,'Area_3', 'Area_4', 'Area_6']
test_areas =['Area_5']

dataset_path = './s3dis/'
model_path = './3D-BoNet/model/all_ep10/model.cptk'

In [0]:
net, data = Evaluation.load_net_data(dataset_path, train_areas, test_areas, model_path)

In [9]:
# データセットの読み込み
from helper_data_s3dis import Data_S3DIS as Data

data = Data(dataset_path, train_areas, test_areas, train_batch_size=4)

check area: Area_1
check area: Area_2
check area: Area_3
check area: Area_4
check area: Area_6
check area: Area_5
train files: 16733
test files: 6852


In [10]:
from main_train import train
train(net, data, eval=False, start_ep = 10, ep_num = 10)

ストリーミング出力は最後の 5000 行に切り捨てられました。
 [23 22 21 20 14 19  0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 17 18]
 [23 22 17  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21]
 [23 22 17 20 21 19  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18]]
zep 0 i 2210 psemce 0.3053888 bbvert -0.059085876 l2 0.16308331 ce 0.30974117 siou -0.53191036 bbscore 0.11320881 pmask 0.43288675
zep 0 i 2220 psemce 0.7931558 bbvert 0.019856483 l2 0.14239588 ce 0.33792162 siou -0.46046102 bbscore 0.18368115 pmask 0.7248684
zep 0 i 2230 psemce 0.57340646 bbvert 0.078261346 l2 0.1593504 ce 0.32875717 siou -0.40984622 bbscore 0.13928162 pmask 0.73454636
zep 0 i 2240 psemce 1.2096612 bbvert 0.32088232 l2 0.13663007 ce 0.4599331 siou -0.27568084 bbscore 0.29602423 pmask 0.82865596
zep 0 i 2250 psemce 0.46277478 bbvert 0.17630363 l2 0.23101628 ce 0.38534248 siou -0.44005513 bbscore 0.23184234 pmask 0.640646
zep 0 i 2260 psemce 0.6448736 bbvert 0.057557076 l2 0.20075247 ce 0.3311407 siou -0.47433606 bbs